In [1]:
1+1

2

In [2]:
import os
%pwd

'e:\\MLOPS\\TextSummarizer\\research'

In [3]:
os.chdir('../')
%pwd

'e:\\MLOPS\\TextSummarizer'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [37]:
from src.textSummarizer.constants import*
from src.textSummarizer.utils.common import read_yaml,create_directories

In [38]:
class ConfigurationManager:
    def __init__(self, 
                 config_path=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        
        
    
                

In [39]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger


### COMPONENTS

In [40]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"File downloaded successfully")
        else:
            logger.info("File already exists")

    
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
          


In [41]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-09-06 13:40:28,727: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-06 13:40:28,731: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-06 13:40:28,734: INFO: common]: Created directory at: artifacts
[2025-09-06 13:40:28,735: INFO: common]: Created directory at: artifacts/data_ingestion
[2025-09-06 13:41:05,396: INFO: 3505038015]: File downloaded successfully
